In [1]:
import os
import pickle
import sqlite3
import numpy as np
import pandas as pd
import ast  #to convert string to list
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity


### **Text Preprocessing**

**LOAD DATA**

In [2]:
movies = pd.read_csv('../archive/tmdb_5000_movies.csv')
credits = pd.read_csv('../archive/tmdb_5000_credits.csv')

In [3]:
print(movies.shape)
movies.head(2)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",19-05-2007,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [4]:
print(credits.shape)
credits.head(2)

(4803, 4)


,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [5]:
movies = movies.merge(credits, on="title")
print(movies.shape)
movies.head(2)

(4808, 23)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [6]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

**CREATE DFS**

In [7]:
movie_rec_df = movies[['genres', 'id', 'keywords', 'title', 'overview', 'cast', 'crew']]
reference_df = movies[['id', 'title', 'status', 'release_date']]
numeric_filters_df = movies[['id', 'budget', 'popularity', 'vote_average', 'vote_count', 'revenue', 'runtime']]
genrekeywords_filters_df = movies[['id', 'genres', 'keywords']]
castcrew_filters_df = movies[['id', 'cast', 'crew']]
rating_df = movies[['vote_average', 'vote_count', 'popularity']]


In [8]:
movie_rec_df.head(1)

,genres,id,keywords,title,overview,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


**MISSING AND DUPLICATE VALUES**

In [9]:
movies.isnull().sum()

budget                     0
genres                     0
homepage                3095
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
movie_id                   0
cast                       0
crew                       0
dtype: int64

In [10]:
# Fill 'homepage' column with "https://#"
movies['homepage'].fillna('https://#', inplace=True)

# Remove null value rows for 'overview', 'release_date', and 'runtime' columns
movies.dropna(subset=['overview', 'release_date', 'runtime'], inplace=True)

# Fill 'tagline' column null values with corresponding 'overview' value
movies['tagline'].fillna(movies['overview'], inplace=True)

movies.isnull().sum()

budget                  0
genres                  0
homepage                0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
tagline                 0
title                   0
vote_average            0
vote_count              0
movie_id                0
cast                    0
crew                    0
dtype: int64

In [11]:
movies.reset_index(drop=True, inplace=True)

In [12]:
movies.duplicated().sum()

0

**COLUMN FORMAT PREPROCESSING TO CREATE PARAGRAPH FOR `tags` COLUMN**

In [13]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [14]:
# Have : '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
# Need : ["Action", "Adventure", "Fantasy", "Science Fiction"]

def extract_listof_tags(argobj):
    # convert the given string of list to actual list
    argobj = ast.literal_eval(argobj)
    # initialize empty list
    L = []
    # run loop through the list of dict
    for dictele in argobj:
        # to remove white-space from the string
        # i.e., to make "Abc Def" -> "AbcDef"
        tagname = dictele["name"].replace(" ", "")
        # append the tagname to list L
        L.append(tagname)
    return L
movies['genres'] = movies['genres'].apply(extract_listof_tags)
movies['keywords'] = movies['keywords'].apply(extract_listof_tags)

In [15]:
movies['cast'][0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [16]:
def extract_3_cast_names(argobj):
    # convert the given string of list to actual list
    argobj = ast.literal_eval(argobj)
    # initialize empty list
    L = []
    # run loop through the list of dict
    for i in range(3):
        try:
            dictele = argobj[i]
            # to remove white-space from the string
            # i.e., to make "Abc Def" -> "AbcDef"
            castname = dictele["name"].replace(" ", "")
            # append the castname to list L
            L.append(castname)
        except:
            pass
    return L
def extract_director_name(argobj):
    # convert the given string of list to actual list
    argobj = ast.literal_eval(argobj)
    # initialize empty list
    L = []
    # run loop through the list of dict
    for dictele in argobj:
        if dictele['job']=='Director':
            # to remove white-space from the string
            # i.e., to make "Abc Def" -> "AbcDef"
            directorname = dictele["name"].replace(" ", "")
            # append the directorname to list L
            L.append(directorname)
            break
    return L

movies['cast'] = movies['cast'].apply(extract_3_cast_names)
movies['crew'] = movies['crew'].apply(extract_director_name)
movies['crew']

0           [JamesCameron]
1          [GoreVerbinski]
2              [SamMendes]
3       [ChristopherNolan]
4          [AndrewStanton]
               ...        
4799     [RobertRodriguez]
4800         [EdwardBurns]
4801          [ScottSmith]
4802          [DanielHsia]
4803     [BrianHerzlinger]
Name: crew, Length: 4804, dtype: object

In [17]:
movies['overview'] = movies['overview'].apply(lambda x : x.split())
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[Action, Adventure, Fantasy, ScienceFiction]",http://www.avatarmovie.com/,19995,"[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [18]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4804 entries, 0 to 4803
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4804 non-null   int64  
 1   genres                4804 non-null   object 
 2   homepage              4804 non-null   object 
 3   id                    4804 non-null   int64  
 4   keywords              4804 non-null   object 
 5   original_language     4804 non-null   object 
 6   original_title        4804 non-null   object 
 7   overview              4804 non-null   object 
 8   popularity            4804 non-null   float64
 9   production_companies  4804 non-null   object 
 10  production_countries  4804 non-null   object 
 11  release_date          4804 non-null   object 
 12  revenue               4804 non-null   int64  
 13  runtime               4804 non-null   float64
 14  spoken_languages      4804 non-null   object 
 15  status               

**NOW, CREATE `tags` COLUMN MERGING `genres, keywords, overview, cast, crew` COLUMNS AND SOME OTHER DFS**

In [19]:
movies['tags'] = movies['tags'].apply(lambda x : " ".join(x).lower())
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew,tags
0,237000000,"[Action, Adventure, Fantasy, ScienceFiction]",http://www.avatarmovie.com/,19995,"[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"in the 22nd century, a paraplegic marine is di..."


In [20]:
movie_rec_df = movies[['id', 'title', 'tags']]
reference_df = movies[['id', 'title', 'status', 'release_date']]
numeric_filters_df = movies[['id', 'budget', 'popularity', 'vote_average', 'vote_count', 'revenue', 'runtime']]
genrekeywords_filters_df = movies[['id', 'genres', 'keywords']]
castcrew_filters_df = movies[['id', 'cast', 'crew']]
rating_df = movies[['vote_average', 'vote_count', 'popularity']]


In [21]:
movie_rec_df.head(1)

,id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."


In [22]:
numeric_filters_df.head(1)

,id,budget,popularity,vote_average,vote_count,revenue,runtime
0,19995,237000000,150.437577,7.2,11800,2787965087,162.0


In [23]:
numeric_filters_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4804 entries, 0 to 4803
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4804 non-null   int64  
 1   budget        4804 non-null   int64  
 2   popularity    4804 non-null   float64
 3   vote_average  4804 non-null   float64
 4   vote_count    4804 non-null   int64  
 5   revenue       4804 non-null   int64  
 6   runtime       4804 non-null   float64
dtypes: float64(3), int64(4)
memory usage: 262.8 KB


**STEMMING**

In [24]:
ps = PorterStemmer()
def stemparagraph(paragraph):
    res = []
    for word in paragraph.split():
        res.append(ps.stem(word))
    return " ".join(res)
movie_rec_df['tags'] = movie_rec_df['tags'].apply(stemparagraph)

C:\Users\Rakesh\AppData\Local\Temp\ipykernel_15016\374745880.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_rec_df['tags'] = movie_rec_df['tags'].apply(stemparagraph)


In [25]:
movie_rec_df['tags'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

**VECTORIZATION OF TAGS PARAGRAPH**

In [26]:
ct_vec = CountVectorizer(max_features=5000, stop_words='english')
vectors = ct_vec.fit_transform(movie_rec_df['tags']).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

**CALCULATE COSINE DISTANCE**

In [27]:
similarity_score = cosine_similarity(vectors)
similarity_score[0]

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [28]:
sorted(list(enumerate(similarity_score[0])), reverse=True, key=lambda x : x[1])[1:6]

[(1216, 0.28676966733820225),
 (2409, 0.26901379342448517),
 (3730, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.25038669783359574)]

In [29]:
def recommend(movie):
    movieindx = movie_rec_df[ movie_rec_df['title']==movie ].index[0]
    distances = similarity_score[movieindx]
    recommandationlist = sorted(list(enumerate(distances)), reverse=True, key=lambda x : x[1])[1:6]
    for rec_movie in recommandationlist:
        print(movie_rec_df.iloc[rec_movie[0]].title)

In [30]:
recommend('The Prestige') # `Batman` `Avatar` `Pride and Prejudice and Zombies`

The Number 23
Enter Nowhere
Eye of the Beholder
The Spanish Prisoner
Final Destination 3


**SAVE MODEL PKL FILE**

In [31]:
target_dir = '../models'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

In [32]:
pickle.dump(movie_rec_df.to_dict(), open('../models/moviesdict.pkl', 'wb'))

In [33]:
pickle.dump(similarity_score, open('../models/simimat.pkl', 'wb'))

### **Testing DB**

In [34]:
# Importing necessary modules
import pandas as pd
import sqlite3
import os
from itertools import product

# Function to create an SQLite database with specified tables based on the DataFrame
def create_multiple_tables(df, table_specs, db_name="test.db"):
    # Ensure the target directory exists
    target_dir = '../created-dbs'
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Full path to the database
    full_db_path = os.path.join(target_dir, db_name)

    # Connect to SQLite database (or create it)
    conn = sqlite3.connect(full_db_path)

    # Loop through each table specification
    for table_name, columns in table_specs.items():
        # Identify the special columns (that contain lists)
        special_columns = [col for col in columns if isinstance(df[col].iloc[0], list)]

        # If there are no special columns, create the table directly
        if not special_columns:
            df[columns].to_sql(table_name, conn, if_exists="replace", index=False)
        else:
            # Create an empty list to store the expanded rows
            expanded_data = []
            
            # Loop through each row of the DataFrame
            for idx, row in df.iterrows():
                # Create Cartesian product for each special column
                values_for_cartesian_product = [row[col] for col in special_columns]
                cartesian_product = list(product(*values_for_cartesian_product))
                
                # Create a new row for each combination in the Cartesian product
                for combination in cartesian_product:
                    new_row = {col: row[col] for col in columns if col not in special_columns}
                    new_row.update({col: val for col, val in zip(special_columns, combination)})
                    expanded_data.append(new_row)

            # Create a new DataFrame from the expanded data
            expanded_df = pd.DataFrame(expanded_data)

            # Create the table
            expanded_df.to_sql(table_name, conn, if_exists="replace", index=False)

    # Commit and close the connection
    conn.commit()
    conn.close()

    print(f"Database {db_name} has been created at {full_db_path}.")


# Sample DataFrame
sample_data = {
    'A': ['a1', 'a2', 'a3'],
    'B': ['b1', 'b2', 'b3'],
    'C': ['c1', 'c2', 'c3'],
    'D': [['d1', 'd2', 'd3'], ['d2', 'd3'], ['d1']],
    'E': [['e1', 'e2'], ['e2', 'e3'], ['e1', 'e2', 'e3']],
    'F': ['f1', 'f2', 'f3'],
    'G': [['d1', 'd2', 'd3'], ['d2', 'd3'], ['d1']],
    'H': [['e1', 'e2'], ['e2'], ['e1', 'e2', 'e3']],
    'I': ['i1', 'i2', 'i3'],
    'J': ['j1', 'j2', 'j3']
}
sample_df = pd.DataFrame(sample_data)

# Table specifications
table_specs = {
    'table1': ['A', 'B', 'C'],
    'table2': ['A', 'D', 'E'],
    'table3': ['A', 'G', 'H']
}

# Create SQLite database with multiple tables
create_multiple_tables(sample_df, table_specs)


Database test.db has been created at ../created-dbs\test.db.


In [35]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew,tags
0,237000000,"[Action, Adventure, Fantasy, ScienceFiction]",http://www.avatarmovie.com/,19995,"[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"in the 22nd century, a paraplegic marine is di..."
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drugabuse, exoticisland, eastindiatrad...",en,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"captain barbossa, long believed to be dead, ha..."


### **Create DB**

In [36]:
# Function to create an SQLite database with specified tables based on the DataFrame
def create_multiple_tables(df, table_specs, db_name="movies_database.db"):
    # Ensure the target directory exists
    target_dir = '../created-dbs'
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Full path to the database
    full_db_path = os.path.join(target_dir, db_name)

    # Connect to SQLite database (or create it)
    conn = sqlite3.connect(full_db_path)

    # Loop through each table specification
    for table_name, columns in table_specs.items():
        # Identify the special columns (that contain lists)
        special_columns = [col for col in columns if isinstance(df[col].iloc[0], list)]

        # If there are no special columns, create the table directly
        if not special_columns:
            df[columns].to_sql(table_name, conn, if_exists="replace", index=False)
        else:
            # Create an empty list to store the expanded rows
            expanded_data = []
            
            # Loop through each row of the DataFrame
            for idx, row in df.iterrows():
                # Create Cartesian product for each special column
                values_for_cartesian_product = [row[col] for col in special_columns]
                cartesian_product = list(product(*values_for_cartesian_product))
                
                # Create a new row for each combination in the Cartesian product
                for combination in cartesian_product:
                    new_row = {col: row[col] for col in columns if col not in special_columns}
                    new_row.update({col: val for col, val in zip(special_columns, combination)})
                    expanded_data.append(new_row)

            # Create a new DataFrame from the expanded data
            expanded_df = pd.DataFrame(expanded_data)

            # Create the table
            expanded_df.to_sql(table_name, conn, if_exists="replace", index=False)

    # Commit and close the connection
    conn.commit()
    conn.close()

    print(f"Database {db_name} has been created at {full_db_path}.")



# Table specifications
table_specs = {
    'movie_rec_df': ['id', 'title', 'tags'],
    'reference_df': ['id', 'title', 'status', 'release_date'],
    'numeric_filters_df': ['id', 'budget', 'popularity', 'vote_average', 'vote_count', 'revenue', 'runtime'],
    'genrekeywords_filters_df': ['id', 'genres', 'keywords'],
    'castcrew_filters_df': ['id', 'cast', 'crew'],
    'rating_df': ['vote_average', 'vote_count', 'popularity'],

}

# Create SQLite database with multiple tables
create_multiple_tables(movies, table_specs)


Database movies_database.db has been created at ../created-dbs\movies_database.db.
